# Naive Bayes and Sentiment Classification

## Problem 1

**Build a naive bayes sentiment classifier using add-1 smoothing, as described in the lecture notes (not binary naive bayes, regular naive bayes). Add an unknown word UNK, as a separate word with count 0. Here is our training corpus:**

In [1]:
# Training Set
negatives = ["just plain boring",
             "entirely predictable and lacks energy",
             "no surprises and very few laughs"]
positives = ["very powerful",
             "the most fun film of the summer"]
documents = {'negative': negatives,
             'positive': positives}
classes = documents.keys()

In [2]:
# Testing Set
test_sentence = "predictable with no originality"

In [3]:
# Set unknown word token
UNK = 'UNK'

**Compute the prior for the two classes + and -, and the likelihoods for each word given the class.**

In [4]:
def tokenize(document):
    """
    Helper function to tokenize a string. I made
    this a function so we can swap out our tokenization
    function in the future if we want to play around with
    things.
    """
    tokens = document.lower().split()  # lowercase, split on whitespace
    return tokens

In [5]:
def count(w, c, D):
    """
    helper function to count occurrences
    of token w in documents D of class c
    
    uses add-1 smoothing
    """
    n = 0
    for d in D[c]:
        tokens = tokenize(d)
        n += tokens.count(w)  # list-counting method
    return (n + 1)  # add-1 smoothing

In [6]:
def train_naive_bayes(D, C):
    """
    train a naive bayes model. 
    The conditional probability of each token
    in the vocabulary of our training set is
    calculated, as well as priors
    """
    global tokenize, count
    
    # Find vocabulary - unique tokens in training set
    vocabulary = set()
    for c in classes:
        for d in documents[c]:
            tokens = tokenize(d)
            for token in tokens:
                vocabulary.add(token)

    vocabulary.add(UNK)  # unknown word, will always have 0 count
    
    priors = dict()
    likelihoods = dict()
    
    # Count total number of documents in training set
    N_doc = 0
    for c in classes:
        N_doc += len(documents[c])

    # Compute priors and likelihoods
    for c in C:

        # Compute prior for this class
        c_documents_list = D[c]
        N_c = len(c_documents_list)
        priors[c] = N_c / N_doc
        
        # sum of count(w,c) for all words in the vocabulary
        c_vocab_total_count = sum([count(w,c,D) for w in vocabulary])
        

        for token in vocabulary:
            likelihoods[(token, c)] = count(token, c, D) / c_vocab_total_count
            
    return (priors, likelihoods, vocabulary)

priors, liks, vocab = train_naive_bayes(documents, classes)
print(priors)
print(liks)

{'negative': 0.6, 'positive': 0.4}
{('film', 'positive'): 0.06666666666666667, ('the', 'negative'): 0.02857142857142857, ('surprises', 'positive'): 0.03333333333333333, ('UNK', 'negative'): 0.02857142857142857, ('of', 'positive'): 0.06666666666666667, ('powerful', 'negative'): 0.02857142857142857, ('laughs', 'positive'): 0.03333333333333333, ('no', 'positive'): 0.03333333333333333, ('most', 'positive'): 0.06666666666666667, ('predictable', 'positive'): 0.03333333333333333, ('boring', 'positive'): 0.03333333333333333, ('just', 'negative'): 0.05714285714285714, ('predictable', 'negative'): 0.05714285714285714, ('energy', 'negative'): 0.05714285714285714, ('plain', 'positive'): 0.03333333333333333, ('fun', 'positive'): 0.06666666666666667, ('lacks', 'negative'): 0.05714285714285714, ('lacks', 'positive'): 0.03333333333333333, ('summer', 'positive'): 0.06666666666666667, ('few', 'positive'): 0.03333333333333333, ('few', 'negative'): 0.05714285714285714, ('energy', 'positive'): 0.0333333333

**Then compute whether the sentence in the test set is of class positive or negative. Make sure you know the correct Bayes equation to use to compute a value for each class in order to answer this question.**

In [7]:
def test_naive_bayes(test_sent, priors, liks, vocabulary, C):
    """
    test_tokens is a list of tokens
    preprocessed (for UNK replacement etc.)
    """
    
    # replace unknown test set words with UNK token
    test_tokens = tokenize(test_sent)

    for i in range(len(test_tokens)):
        if test_tokens[i] not in vocabulary:
            test_tokens[i] = UNK
    
    # Compute class probabilities for this string of tokens
    class_probabilities = dict()
    
    for c in C:
        class_probabilities[c] = priors[c]

        for token in test_tokens:
            class_probabilities[c] *= liks[(token, c)]
            
    print(class_probabilities)
    return max(class_probabilities, key=class_probabilities.get)  # argmax

print(test_naive_bayes(test_sentence, priors, liks, vocab, classes))

{'negative': 1.599333610995418e-06, 'positive': 4.938271604938272e-07}
negative


**What would the answer be without add-1 smoothing?**

In [8]:
def count(w, c, D):
    """
    the power of modularity.
    redefine our helper count function,
    to do the same thing but not use add-1
    smoothing.
    
    count occurrences
    of token w in class c
    """
    n = 0
    for d in D[c]:
        tokens = tokenize(d)
        n += tokens.count(w)  # list-counting method
    return n

priors, liks, vocab = train_naive_bayes(documents, classes)
print(test_naive_bayes(test_sentence, priors, liks, vocab, classes))

{'negative': 0.0, 'positive': 0.0}
negative


Without add-1 smoothing, the unknown words make the probability for any class 0.

**Would using binary multinomial Naive Bayes change anything?**

No, because changing the counts to 0 would still leave the unknown words in the test set with a count of 0, and thus a conditional probability of 0. We either need to use add-1 smoothing, or drop the unknown words entirely from the test sentence.

**What would happen if you used the second alternative method in Section 3.3.1 of J&M to determine the count of UNK?**

The second method mentioned is to set some amount of words with occur infrequently in the training set, to UNK, and then compute the count of UNK normally.

In [9]:
# count words
token_counts = {}
for c in classes:
    for d in documents[c]:
        for token in tokenize(d):
            token_counts[token] = token_counts.get(token, 0) + 1
            
print(token_counts)

{'energy': 1, 'lacks': 1, 'the': 2, 'boring': 1, 'fun': 1, 'and': 2, 'powerful': 1, 'no': 1, 'summer': 1, 'film': 1, 'laughs': 1, 'surprises': 1, 'very': 2, 'entirely': 1, 'of': 1, 'few': 1, 'plain': 1, 'most': 1, 'just': 1, 'predictable': 1}


It turns out, most words occur infrequently in our training set. If we set tokens with count less than 2 to UNK, we would end up removing most of the useful information from our model. So we won't do that.

## Problem 2

**We are given the following corpus, modified from the one in the chapter:**

**&lt;s&gt; I am Sam &lt;/s&gt;**

**&lt;s&gt; Sam I am &lt;/s&gt;**

**&lt;s&gt; I am Sam &lt;/s&gt;**

**&lt;s&gt; I do not like green eggs and Sam &lt;/s&gt;**

**If we use linear interpolation smoothing between a maximum-likelihood bi-gram model and a maximum-likelihood unigram model with $\lambda_1 = \frac{1}{2}$ and $\lambda_2 = \frac{1}{2}$, what is $P(Sam \mid am)$? Include &lt;s&gt; and &lt;/s&gt; in your counts just like any other token.**

In [10]:
from collections import Counter
import numpy as np

# Tokenize training corpus
training_sentences = ['<s> I am Sam </s>',
                      '<s> Sam I am </s>',
                      '<s> I am Sam </s>',
                      '<s> I do not like green eggs and Sam </s>']

# Let's use a basic split for our tokenization in this case, because we want <s> to be one token.


training_tokens = [tokenize(sent) for sent in training_sentences]

# Build unigram and bigram counts
unigram_counts = Counter()
bigram_counts = Counter()

for token_list in training_tokens:
    unigram_counts += Counter(token_list)
    bigram_counts += Counter(zip(token_list[:-1], token_list[1:]))

# Normalize counts to get MLE models
unigram_model = dict(unigram_counts)
bigram_model = dict(bigram_counts)

unigram_total_count = sum(unigram_model.values())  # sum all unigram counts

for unigram in unigram_model.keys():
    unigram_model[unigram] /= unigram_total_count
    
for bigram in bigram_model.keys():
    w1 = bigram[0]
    bigram_model[bigram] /= unigram_counts[w1]

# These models are max likelihood estimators, so they aren't necessarily PDFs
# in this case the unigram will be a PDF but the bigram estimate won't be
print("Unigram model: ", unigram_model)
print("Bigram model: ", bigram_model)

Unigram model:  {'i': 0.16, 'like': 0.04, 'not': 0.04, 'am': 0.12, 'green': 0.04, 'sam': 0.16, '<s>': 0.16, 'do': 0.04, 'and': 0.04, 'eggs': 0.04, '</s>': 0.16}
Bigram model:  {('<s>', 'i'): 0.75, ('am', 'sam'): 0.6666666666666666, ('sam', 'i'): 0.25, ('like', 'green'): 1.0, ('sam', '</s>'): 0.75, ('do', 'not'): 1.0, ('not', 'like'): 1.0, ('am', '</s>'): 0.3333333333333333, ('i', 'do'): 0.25, ('eggs', 'and'): 1.0, ('and', 'sam'): 1.0, ('i', 'am'): 0.75, ('green', 'eggs'): 1.0, ('<s>', 'sam'): 0.25}


In [11]:
lambda1 = 1/2
lambda2 = 1/2

p_am_sam = lambda1 * unigram_model['sam'] + lambda2 * bigram_model[('am', 'sam')]
print("P(Sam | am) = %f" % p_am_sam)

P(Sam | am) = 0.413333


## Problem 3

**Suppose we didn’t use the end-symbol &lt;/s&gt;. Train an unsmoothed bigram grammar on the following training corpus without using the end-symbol &lt;/s&gt;:**

**&lt;s&gt; a b**

**&lt;s&gt; b b**

**&lt;s&gt; b a**

**&lt;s&gt; a a**

In [12]:
# Tokenize training corpus
training_sentences = ['<s> a b',
                      '<s> b b',
                      '<s> b a',
                      '<s> a a']

training_tokens = [tokenize(sent) for sent in training_sentences]

# Build unigram and bigram counts
unigram_counts = Counter()
bigram_counts = Counter()

for token_list in training_tokens:
    unigram_counts += Counter(token_list)
    bigram_counts += Counter(zip(token_list[:-1], token_list[1:]))

# Normalize bigram counts to get MLE model
bigram_model = dict(bigram_counts)
    
for bigram in bigram_model.keys():
    w1 = bigram[0]
    bigram_model[bigram] /= unigram_counts[w1]

print("Bigram model: ", bigram_model)

Bigram model:  {('a', 'b'): 0.25, ('b', 'b'): 0.25, ('b', 'a'): 0.25, ('<s>', 'a'): 0.5, ('a', 'a'): 0.25, ('<s>', 'b'): 0.5}


**Demonstrate that your bigram model does not assign a single probability distribution across all sentence lengths by showing that the sum of the probability of the four possible 2 word sentences over the alphabet {a,b} is 1.0, and the sum of the probability of all possible 3 word sentences over the alphabet {a,b} is also 1.0.**

In [13]:
import itertools

alphabet = ['a', 'b']

# produce cartesian product of alphabet with itself
possible_two_word_sentences = itertools.product(alphabet, repeat=2)

prob_sum = 0
for sent in possible_two_word_sentences:
    sent_prob = bigram_model[sent]

    prob_sum += sent_prob
    print(sent, ":", sent_prob)
    
print("Sum: %.2f" % prob_sum)

('a', 'a') : 0.25
('a', 'b') : 0.25
('b', 'a') : 0.25
('b', 'b') : 0.25
Sum: 1.00


In [14]:
possible_three_word_sentences = itertools.product(alphabet, repeat=3)

prob_sum = 0
for sent in possible_three_word_sentences:
    bigram1 = sent[:2]
    bigram2 = sent[1:]
    
    sent_prob = bigram_model[bigram1] * bigram_model[bigram2]

    prob_sum += sent_prob
    print(sent, ":", sent_prob)
    
print("Sum: %.2f" % prob_sum)

('a', 'a', 'a') : 0.0625
('a', 'a', 'b') : 0.0625
('a', 'b', 'a') : 0.0625
('a', 'b', 'b') : 0.0625
('b', 'a', 'a') : 0.0625
('b', 'a', 'b') : 0.0625
('b', 'b', 'a') : 0.0625
('b', 'b', 'b') : 0.0625
Sum: 0.50


The second sum is not 1.0, but the point is still made since obviously the sum of probabilities over all sentence lengths will be > 1.0.

## Problem 4

**A robot, which only has a camera as a sensor, can either be in one of two locations: L1 or L2. The robot doesn’t know exactly where it is and it represents this uncertainty by keeping track of two probabilities: P(L1) and P(L2). Based on all past observations, the robot thinks that there is a 0.8 probability it is in L1 and a 0.2 probability that it is in L2.**

**The robot’s vision algorithm detects a window, and although there is only a window in L2, it can’t conclude that it is in fact in L2: its image recognition algorithm is not perfect. The probability of observing a window given there is no window at its location is 0.2 and the probability of observing a window given there is a window is 0.9. After incorporating the observation of a window, what is the robot’s new values for P(L1) and P(L2)?**

By Bayes Rule, $$P(LX \mid window) = \frac{P(window \mid LX) * P(LX)}{P(window)}$$

We can use the law of total probability: $$P(Y) = \sum_\limits{X \in \Omega} P(Y | X) * P(X)$$ to find $P(window)$.

In [15]:
prior_l1 = 0.8
prior_l2 = 0.2

p_window_l1 = 0.2
p_window_l2 = 0.9

# law of total probability
p_window = p_window_l1 * prior_l1 + p_window_l2 * prior_l2

# bayes rule
posterior_l1 = p_window_l1 * prior_l1 / p_window
posterior_l2 = p_window_l2 * prior_l2 / p_window

print("P(L1) = %f" % posterior_l1)
print("P(L2) = %f" % posterior_l2)

P(L1) = 0.470588
P(L2) = 0.529412


Once again we see how human intuition often fails to grasp bayesian statistics. At least my intuition often does ^^

## Problem 5

**Binary multinomial NB seems to work better on some problems than full count NB, but full count works better on others. For what kinds of problems might binary NB be better, and why? (There is no known right answer to this question, but I'd like you to think about the possibilities.) Come up with an example.**

Binary NB is less sensitive to the details in text. I'm not convinced that's a good thing, though. If an author describes a movie as "fantastic" 5 times in a text, then disregarding negation that seems to me stronger evidence of a positive sentiment than only one instance of "fantastic". I would consider using Binary NB when you have a lack of data, and want a simpler model to ensure it will generalize. Let's look at an example below of where the two models can disagree given the same data.

In [51]:
# Full count with add-1 smoothing
def count(w, c, D):
    n = 0
    for d in D[c]:
        tokens = tokenize(d)
        n += tokens.count(w)
    return (n + 1)  # add-1 smoothing

In [57]:
# This example is inspired by Exercise 4.3
negatives = ["Good poor poor poor",
             "poor great good poor " + \
             "great poor poor POOR",
             "Poor poor"]

positives = ["good good good Great great great",
             "poor great great"]

documents = {'negative': negatives,
             'positive': positives}
classes = documents.keys()

priors, liks, vocab = train_naive_bayes(documents, classes)
print("Priors: ", priors)
print("Likelihoods: ", liks)

Priors:  {'negative': 0.6, 'positive': 0.4}
Likelihoods:  {('great', 'positive'): 0.2857142857142857, ('good', 'positive'): 0.2857142857142857, ('UNK', 'negative'): 0.14285714285714285, ('poor', 'negative'): 0.2857142857142857, ('good', 'negative'): 0.2857142857142857, ('UNK', 'positive'): 0.14285714285714285, ('poor', 'positive'): 0.2857142857142857, ('great', 'negative'): 0.2857142857142857}


In [53]:
test_sent = "A good, good plot and great characters, but poor acting."

print("Full count results: ", test_naive_bayes(test_sent, priors, liks, vocab, classes))

{'negative': 8.557859130511291e-15, 'positive': 2.5356986536167335e-13}
Full count results:  positive


In [58]:
# Binary count with add-1 smoothing
def count(w, c, D):
    for d in D[c]:
        tokens = tokenize(d)
        if w in tokens:
            return 2
    return 1

priors, liks, vocab = train_naive_bayes(documents, classes)
print("Likelihoods: ", liks)

Likelihoods:  {('great', 'positive'): 0.2857142857142857, ('good', 'positive'): 0.2857142857142857, ('UNK', 'negative'): 0.14285714285714285, ('poor', 'negative'): 0.2857142857142857, ('good', 'negative'): 0.2857142857142857, ('UNK', 'positive'): 0.14285714285714285, ('poor', 'positive'): 0.2857142857142857, ('great', 'negative'): 0.2857142857142857}


In [55]:
print("Binary count results: ", test_naive_bayes(test_sent, priors, liks, vocab, classes))

{'negative': 9.908244453806926e-11, 'positive': 6.60549630253795e-11}
Binary count results:  negative


In this particular example, full-count NB and binary NB disagree on the sentiment classification. Does it make sense for repeated adjectives like "good, good" to be considered as valuable information showing emphasis? Perhaps. But in this case ignoring the emphasis allowed the model to side more strongly with its priors and make the correct classification.

Personally though I wouldn't worry about edge cases like this, and would trust full-count NB more given enough data.